In [48]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms, io
from torchvision.transforms import v2
from sklearn.utils import shuffle
from PIL import Image
import matplotlib.pyplot as plt

## Creating Custom Dataset for loading the data to the models

In [4]:
### Paths to the dataframe containing

train_folds = ["/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/train1.csv",
              "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/train2.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/train3.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/train4.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/train5.csv"]

val_folds = ["/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/validation1.csv",
              "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/validation2.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/validation3.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/validation4.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/validation5.csv"]

test_folds = ["/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/test1.csv",
              "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/test2.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/test3.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/test4.csv",
             "/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/Annotation/CSV/test5.csv"]

In [138]:
class DataClass(Dataset):
    
    def __init__(self, annotations, transform = None):
        self.df = pd.read_csv(annotations)
        self.transform = transform
        
    def __getitem__(self, idx):
        img_path = "/".join(["/kaggle/input/covidctset-a-large-covid19-ct-scans-dataset/TrainValidation",self.df.iloc[idx, 0]])
        label = self.df.iloc[idx, 1]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)
        
        return image, label
    
    def __len__(self):
        return len(self.df)

In [131]:
train_transforms = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.RandomRotation(360),
    v2.RandomAffine(degrees=0, translate=(0.05, 0.05), shear=0.05),
    v2.ToTensor()
])

In [147]:
i = 0
TrainDataClass = DataClass(train_folds[i], train_transforms)
ValDataClass = DataClass(val_folds[i])
TestDataClass = DataClass(test_folds[i])

shape=(512,512,1)

In [148]:
TrainDataLoader = DataLoader(TrainDataClass, batch_size = 14, shuffle = True)
ValDataLoader = DataLoader(ValDataClass, batch_size = 10, shuffle = False)
TestDataLoader = DataLoader(TestDataClass, batch_size = 10, shuffle = False)

In [152]:
class FPNModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        